In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import zipfile
from datetime import datetime

In [3]:
# Choose city:
city = 'Detroit'

# Choose starting hour and number of hour to look at/graph
starting_hour = 1
hour = 8760             #choose number of hours to graph

# Choose building id
li = []
li2 = []
for i in list(range(400)):
    building_id = i+1

    # Read resstock results summary folder:
    B = list(range(400))
    if city == 'Detroit':
        load_folder = 'C:\\Users\\atpha\\Downloads\\400 Buildings - HP\\'
    else:
        load_folder = 'C:\\Users\\atpha\\Documents\\Postdocs\\Projects\\TES\\Data\\400_Buildings_EB\\LA_NYC_ORL_SEA_ATL_MIN_PHX\\'

    resstock_results = pd.read_csv(load_folder + 'results.csv')
    resstock_results = resstock_results[resstock_results['building_characteristics_report.location_city'].str.contains(city) == True]
    #resstock_results = resstock_results[(resstock_results['apply_upgrade.run_measure'] == 0) & (resstock_results['apply_upgrade_2.run_measure'] == 0)]
    resstock_results = resstock_results.sort_values(by='build_existing_model.building_id')
    resstock_results['build_existing_model.building_id'] = resstock_results['build_existing_model.building_id'] % 400
    resstock_results.loc[resstock_results['build_existing_model.building_id'] ==0, 'build_existing_model.building_id'] = 400
    resstock_results = resstock_results.reset_index()
    resstock_results = resstock_results.drop(['index'], axis=1)

    resstock_building_id = building_id

    folder_id = resstock_results.loc[resstock_results['build_existing_model.building_id'] == resstock_building_id, '_id']
    folder_id = folder_id.reset_index()
    folder_id = folder_id.drop(['index'], axis=1)
    folder_id = folder_id.iloc[0,0]

    archive = zipfile.ZipFile(load_folder + folder_id + '\\' + 'data_point.zip')

    load_data_path = archive.extract('enduse_timeseries.csv', load_folder + folder_id + '\\')
    load_data_raw = pd.read_csv(load_data_path)
    archive.close()

    # Hourly load
    #load_data_annual = load_data_raw['total_site_electricity_kwh']
    df = pd.DataFrame()
    df['building_id'] = building_id
    df['heating load'] = load_data_raw['electricity_heating_kwh']
    df['fan heating load'] = load_data_raw['electricity_fans_heating_kwh']
    df['building_id'] = building_id

    li.append(df)

heating_load = pd.concat(li, axis=0)
heating_load.to_csv("C:\\Users\\atpha\\Documents\\Postdocs\\Projects\\TES\Data\\fan heating load\\heating_load.csv")

